# Adding OCO-[2|3]COGs

In [1]:
import boto3
import concurrent.futures
import os
from concurrent.futures import ProcessPoolExecutor, as_completed
from osgeo import gdal

gdal.DontUseExceptions()

s3_client = boto3.client("s3")

In [16]:
source_bucket = "sdap-dev-zarr"
source_dir = "OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog"
# Set the number of worker processes
NUM_WORKERS = 4
destination_bucket = "veda-data-store-staging"
collection = "oco2-co2"
prefix = f"{source_dir}/{collection}"
collection_prefix_collection_id_map = {
    "oco3-co2": "oco3-co2-sam-l3-cogs",
    "oco2-co2": "oco2-co2-target-cogs",
    "oco3-sif": "oco3-co2-sif-cogs",
}

In [10]:
# List all objects in the source bucket
all_objects = []
continuation_token = None

while all_objects == [] or continuation_token is not None:
    args = dict(Bucket=source_bucket, Prefix=prefix)
    if continuation_token is not None:
        args["ContinuationToken"] = continuation_token
    response = s3_client.list_objects_v2(**args)
    [all_objects.append(obj["Key"]) for obj in response.get("Contents", [])]
    continuation_token = response.get("NextContinuationToken", None)

In [11]:
len(all_objects)

2158

In [12]:
all_objects[0]

'OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif'

In [21]:
def copy_object(object_key: str):
    copy_source = {"Bucket": source_bucket, "Key": object_key}
    destination_file_name = object_key.split("/")[-1]
    collection_prefix = destination_file_name.split("_")[0]
    destination_dir = collection_prefix_collection_id_map[collection_prefix]
    destination_object_key = f"{destination_dir}/{destination_file_name}"
    s3_client.copy_object(
        CopySource=copy_source, Bucket=destination_bucket, Key=destination_object_key
    )
    print(
        f"Copied: s3://{source_bucket}/{object_key} to s3://{destination_bucket}/{destination_object_key}"
    )


def process_file(object_key: str):
    """Function to process a single file."""
    dataset = gdal.Open(f"/vsis3/{source_bucket}/{object_key}", gdal.GA_ReadOnly)

    if dataset is None:
        return file_path, "Failed to open"

    band = dataset.GetRasterBand(1)
    stats = band.GetStatistics(True, True)

    # Check for errors
    err_type = gdal.GetLastErrorType()
    err_msg = gdal.GetLastErrorMsg()

    if "Failed to compute statistics, no valid pixels found in sampling" in err_msg:
        return dict(
            object_key=object_key,
            error_message=f"GDAL Error: {err_msg}",
            status="error",
        )
    else:
        copy_object(object_key=object_key)
        return dict(object_key=object_key, error_message=None, status="success")

In [22]:
first_object_key = all_objects[0]
process_file(first_object_key)

Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif


Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif.aux.xml.


{'object_key': 'OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif',
 'error_message': None,
 'status': 'success'}

In [31]:
#!aws s3 ls s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/ #| wc -l

In [29]:
def main():
    failures = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        futures = {
            executor.submit(process_file, object_key): object_key
            for object_key in all_objects[0:100]
        }

        for idx, future in enumerate(as_completed(futures)):
            if idx % 100 == 0:
                print(f"processed file {idx}")
            result = future.result()
            if result["status"] == "error":
                object_key = result["object_key"]
                failures.append((object_key, result))
                # print(f"{object_key}: {result['error_message']}")

    # Save results to a log file
    with open("failed_files.log", "w") as f:
        for object_key, result in failures:
            f.write(f"{object_key}\n")


main()

Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-02-10T180729Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2020-02-10T180729Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-02-10T180729Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2020-02-10T180729Z_filtered_xco2.tif
processed file 0
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2019-10-10T190139Z_unfiltered_xco2

Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-02-10T180729Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-02-10T180729Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-05T202130Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save aux

Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-13T171706Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2020-04-13T171706Z_filtered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-13T171706Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2020-04-13T171706Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-17T221620Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2020-04-17T221620Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-17T221621Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2020-04-17T221621Z_filtered_xco2.tif


Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-13T171706Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-17T221620Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-17T221621Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-13T171706Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-02-29T173727Z_unfiltered_xco2.tif.aux.xml.


Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-02-29T173727Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2020-02-29T173727Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-29T173419Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2020-04-29T173419Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2020-04-21T204252Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_cal001_2020-04-21T204252Z_filtered_xco2.tif


...

Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon102_2020-05-10T074756Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon102_2020-05-06T092216Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2019-09-24T125407Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon102_2020-05-06T092216Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2019-09-24T125407Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Una

Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-01-05T080248Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_coccon103_2020-01-05T080248Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-01-05T080303Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_coccon103_2020-01-05T080303Z_filtered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-01-09T062748Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_coccon103_2020-01-09T062748Z_filtered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-03-23T130802Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_coccon103_2020-03-23T130802Z_filtered_x

Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-01-05T080248Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-01-09T062748Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-03-23T130802Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-01-05T080303Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-01-09T062748Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unabl

Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-01-09T062748Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_coccon103_2020-01-09T062748Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-03-23T130802Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_coccon103_2020-03-23T130802Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-03-27T113519Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_coccon103_2020-03-27T113519Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-03-27T113528Z_filtered_xco2.tif to s3://veda-data-store-staging/oco3-co2-sam-l3-cogs/oco3-co2_coccon103_2020-03-27T113528Z_fi

Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-04-08T065843Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_coccon103_2020-04-08T065846Z_filtered_xco2.tif.aux.xml.
